After we saw the following [tweet](https://twitter.com/cecilejanssens/status/1142864439638679554) from [@cecilejanssens](https://twitter.com/cecilejanssens):

<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Why I think (current) polygenic risk scores are just a phase? Because I believe that scientists will find better ways ways to model the role of SNPs in molecular pathways and their combined contribution to disease risk. This cannot be it. <a href="https://t.co/p9nP5hgQV0">pic.twitter.com/p9nP5hgQV0</a></p>&mdash; Cecile Janssens (@cecilejanssens) <a href="https://twitter.com/cecilejanssens/status/1142864439638679554?ref_src=twsrc%5Etfw">June 23, 2019</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

Daniel and I asked how easy it would be to map single nucleotide polymorphisms (SNPs) from a [Bio2BEL](https://github.com/bio2bel) repository like [``bio2bel_phewascatalog``](https://github.com/bio2bel/phewascatalog) to one of the major pathway databases like KEGG, Reactome, and WikiPathways.

In [1]:
import getpass
import os
import sys
import time
from collections import defaultdict, Counter

import bio2bel_kegg
import bio2bel_phewascatalog
import bio2bel_reactome
import bio2bel_wikipathways
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
%matplotlib inline

In [3]:
print(sys.version)

3.7.3 (default, Mar 27 2019, 09:23:39) 
[Clang 10.0.0 (clang-1000.11.45.5)]


In [4]:
print(time.asctime())

Mon Jun 24 15:14:43 2019


In [5]:
print(getpass.getuser())

cthoyt


In [6]:
print(bio2bel_kegg.get_version())
print(bio2bel_reactome.get_version())
print(bio2bel_wikipathways.get_version())

0.2.5-dev
0.2.4-dev
0.2.4-dev


In [7]:
print(bio2bel_phewascatalog.get_version())

0.0.1-dev


Generate the gene to SNP mappings from PheWAS Catalog. This step can be interchanged with dbSNP, GWAS Catalog, or other sources.

In [8]:
phewascatalog_df = bio2bel_phewascatalog.parser.get_df()
phewascatalog_df = phewascatalog_df[phewascatalog_df.gene_name.notna()]
phewascatalog_df.head()

,chromosome,snp,phewas phenotype,cases,p-value,odds-ratio,gene_name,phewas code,gwas-associations
0,19 45395619,rs2075650,Alzheimer's disease,737,5.237000e-28,2.410,TOMM40,290.11,"Alzheimer's disease, Alzheimer's disease bioma..."
1,19 45395619,rs2075650,Dementias,1170,2.409000e-26,2.114,TOMM40,290.10,"Alzheimer's disease, Alzheimer's disease bioma..."
2,6 396321,rs12203592,Actinic keratosis,2505,4.141000e-26,1.691,IRF4,702.10,"Eye color, Hair color, Freckling, Progressive ..."
3,6 26093141,rs1800562,Iron metabolism disorder,40,3.409000e-25,12.270,HFE,275.10,"Mean corpuscular hemoglobin, Glycated hemoglob..."
4,19 45395619,rs2075650,Delirium dementia and amnestic disorders,1566,8.027000e-24,1.841,TOMM40,290.00,"Alzheimer's disease, Alzheimer's disease bioma..."


In [9]:
gene_to_snps = defaultdict(set)
for snp, gene_symbol in phewascatalog_df[['snp', 'gene_name']].values:
    gene_to_snps[gene_symbol].add(snp)
gene_to_snps = dict(gene_to_snps)

In [10]:
# promiscuity of SNPs
Counter(len(v) for v in gene_to_snps.values())

Counter({2: 246, 3: 92, 1: 1393, 8: 2, 4: 26, 5: 10, 6: 5, 7: 1})

## Map Pathways to Genes

In [11]:
def get_pathway_to_gene(manager):
    pathway_to_gene = defaultdict(set)

    for pathway in tqdm(manager.get_all_pathways(), desc='Getting pathways/genes'):
        for protein in pathway.proteins:
            pathway_to_gene[pathway].add(protein.hgnc_symbol)

    return dict(pathway_to_gene)

def get_pathway_to_snp(pathway_to_gene):
    """Combine the mappings to relate pathways to SNPs.
    
    This could be further extended to weight pathway-SNP associations
    by count, or to normalize by the frequency of each SNP being
    mapped to multiple genes.
    """
    pathway_to_snp = defaultdict(set)

    for pathway, gene_symbols in pathway_to_gene.items():
        for gene_symbol in gene_symbols:
            pathway_to_snp[pathway].update(gene_to_snps.get(gene_symbol, set()))

    return dict(pathway_to_snp)

### KEGG

In [12]:
kegg_manager = bio2bel_kegg.Manager()
kegg_pathway_to_gene = get_pathway_to_gene(kegg_manager)
kegg_pathway_to_snp = get_pathway_to_snp(kegg_pathway_to_gene)

In [13]:
kegg_df = pd.DataFrame([
    (pathway.kegg_id, pathway.name, ', '.join(snps))
    for pathway, snps in kegg_pathway_to_snp.items()
], columns=['kegg_id', 'name', 'snps'])
kegg_df.head()

,kegg_id,name,snps
0,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,"rs560887, rs1789924, rs730497, rs7072268, rs28..."
1,path:hsa00020,Citrate cycle (TCA cycle) - Homo sapiens (human),"rs7122539, rs6792584"
2,path:hsa00030,Pentose phosphate pathway - Homo sapiens (human),
3,path:hsa00040,Pentose and glucuronate interconversions - Hom...,"rs6742078, rs4148325, rs11892031, rs887829"
4,path:hsa00051,Fructose and mannose metabolism - Homo sapiens...,"rs9378688, rs7072268"


### WikiPathways

In [14]:
wikipathways_manager = bio2bel_wikipathways.Manager()
wikipathways_pathway_to_gene = get_pathway_to_gene(wikipathways_manager)
wikipathways_pathway_to_snp = get_pathway_to_snp(wikipathways_pathway_to_gene)

In [15]:
wikipathways_df = pd.DataFrame([
    (pathway.wikipathways_id, pathway.name, ', '.join(snps))
    for pathway, snps in wikipathways_pathway_to_snp.items()
], columns=['wikipathways_id', 'name', 'snps'])
wikipathways_df.head()

,wikipathways_id,name,snps
0,WP23,B Cell Receptor Signaling Pathway,"rs3780792, rs13017599, rs1366594, rs12203592, ..."
1,WP2333,Trans-sulfuration pathway,rs6586282
2,WP2509,Nanoparticle triggered autophagic cell death,"rs7111341, rs2241880, rs3792109, rs891088, rs2..."
3,WP3891,Benzene metabolism,
4,WP1604,Codeine and Morphine Metabolism,"rs6742078, rs4149081, rs887829, rs4148325, rs1..."


### Reactome

In [16]:
reactome_manager = bio2bel_reactome.Manager()
reactome_pathway_to_gene = get_pathway_to_gene(reactome_manager)
reactome_pathway_to_snp = get_pathway_to_snp(reactome_pathway_to_gene)

In [17]:
reactome_df = pd.DataFrame([
    (pathway.reactome_id, pathway.name, ', '.join(snps))
    for pathway, snps in reactome_pathway_to_snp.items()
], columns=['reactome_id', 'name', 'snps'])
reactome_df.head()

,reactome_id,name,snps
0,R-HSA-1640170,Cell Cycle,"rs2277339, rs2312147, rs4668356, rs2244012, rs..."
1,R-HSA-109581,Apoptosis,"rs2084385, rs242557, rs710521, rs8070723, rs81..."
2,R-HSA-1500931,Cell-Cell communication,"rs4925189, rs4468878, rs3784962, rs13078807, r..."
3,R-HSA-8953897,Cellular responses to external stimuli,"rs12138950, rs1127065, rs2244012, rs7805747, r..."
4,R-HSA-4839726,Chromatin organization,"rs2900333, rs7538876, rs12423712, rs10758669, ..."


Output all TSVs for reuse

In [18]:
kegg_df.to_csv('kegg_to_snp.tsv', index=False, sep='\t')
wikipathways_df.to_csv('wikipathways_to_snp.tsv', index=False, sep='\t')
reactome_df.to_csv('reactome_to_snp.tsv', index=False, sep='\t')